# Applied Programming Coding Challenge #2

# General Information

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image
from datetime import datetime
import urllib.request

try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

# 0 Initialize device

In [1]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 1 Load data

## 1.1 Load data from internet

In [ ]:
dir_pokemon_sprites = "../data/pokemon-sprites"

def download_file(url, file_name):
    """Downloads a file from a given URL using to arbitrary headers"""
    
    hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}
    req = urllib.request.Request(url, headers=hdr)
    response = urllib.request.urlopen(req)
    
    # Write response to file
    with open(file_name, mode="wb") as d:
        d.write(response.read())

In [ ]:
def download_sprites_151():
    """Download sprites of 150 pokemon from alexonsager.net"""
    
    pokemon_count = 151
    for i in range(1, pokemon_count+1):
        url = "https://images.alexonsager.net/pokemon/" + str(i) + ".png"
        file_name = dir_pokemon_sprites + "/raw/" + str(i).zfill(3) + ".png"
        download_file(url, file_name)

In [ ]:
# download_sprites_151()

## 1.2 Load sprites into notebook

In [ ]:
image_size = 80
image_dimension = image_size**2

transform = transforms.Compose([
    transforms.Resize(image_size, image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.ImageFolder(dir_pokemon_sprites, transform=transform)

In [ ]:
batch_size = 10
num_workers = 2

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True, 
                                           num_workers=num_workers)

In [ ]:
class Generator(nn.Module):
    def __init__(self, g_input_dim, g_output_dim):
        super(Generator, self).__init__()       
        self.fc1 = nn.Linear(g_input_dim, 256)
        self.fc2 = nn.Linear(self.fc1.out_features, self.fc1.out_features*2)
        self.fc3 = nn.Linear(self.fc2.out_features, self.fc2.out_features*2)
        self.fc4 = nn.Linear(self.fc3.out_features, g_output_dim)
    
    def forward(self, x): 
        x = F.leaky_relu(self.fc1(x), 0.2)
        x = F.leaky_relu(self.fc2(x), 0.2)
        x = F.leaky_relu(self.fc3(x), 0.2)
        return torch.tanh(self.fc4(x))
    

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, d_input_dim):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(d_input_dim, 1024)
        self.fc2 = nn.Linear(self.fc1.out_features, self.fc1.out_features//2)
        self.fc3 = nn.Linear(self.fc2.out_features, self.fc2.out_features//2)
        self.fc4 = nn.Linear(self.fc3.out_features, 1)
    
    def forward(self, x):
        x = F.leaky_relu(self.fc1(x), 0.2)
        x = F.dropout(x, 0.3)
        x = F.leaky_relu(self.fc2(x), 0.2)
        x = F.dropout(x, 0.3)
        x = F.leaky_relu(self.fc3(x), 0.2)
        x = F.dropout(x, 0.3)
        return torch.sigmoid(self.fc4(x))
    

In [ ]:


G = Generator(g_input_dim = image_dimension, g_output_dim = image_dimension).to(device)
D = Discriminator(image_dimension).to(device)

In [ ]:
G

In [ ]:
D

# 2 Define optimizer

In [ ]:
# loss
criterion = nn.BCELoss() 

# optimizer
learning_rate = 0.0002 
G_optimizer = optim.Adam(G.parameters(), lr = learning_rate)
D_optimizer = optim.Adam(D.parameters(), lr = learning_rate)

# 3 Training

## 3.1 Define discriminator training

In [ ]:
def D_train(x):
    D.zero_grad()

    x_real, y_real = x.view(-1, image_dimension), torch.ones(batch_size, 1)
    
    print(x_real)
    print(y_real)
    
    x_real, y_real = Variable(x_real.to(device)), Variable(y_real.to(device))

    D_output = D(x_real)
    
    print(len(D_output))
    print(len(y_real))
    
    D_real_loss = criterion(D_output, y_real)
    D_real_score = D_output

    z = Variable(torch.randn(batch_size, image_dimension).to(device))
    x_fake, y_fake = G(z), Variable(torch.zeros(batch_size, 1).to(device))

    D_output = D(x_fake)
    D_fake_loss = criterion(D_output, y_fake)
    D_fake_score = D_output

    # gradient backprop & optimize ONLY D's parameters
    D_loss = D_real_loss + D_fake_loss
    D_loss.backward()
    D_optimizer.step()
        
    return  D_loss.data.item()

## 3.2 Define generator training

In [ ]:
def G_train(x):
    G.zero_grad()

    z = Variable(torch.randn(batch_size, image_dimension).to(device))
    y = Variable(torch.ones(batch_size, 1).to(device))

    G_output = G(z)
    D_output = D(G_output)
    G_loss = criterion(D_output, y)

    G_loss.backward()
    G_optimizer.step()
        
    return G_loss.data.item()

## 3.3 Train GAN

In [ ]:
# Generic Google Drive mount
if IN_COLAB:
    drive.mount('/content/gdrive')

In [ ]:
def saveFileToGoogleDrive(image, fileName, dirName = "Colab Files"):
    """Saves a file to Google Drive"""
    
    save_image(image, '/content/gdrive/My Drive/' + dirName + '/' + fileName)

def saveFileToLocal(image, fileName, dirName = "mnist"):
    """Saves a file to local directory"""
    
    save_image(image, '../build/' + dirName + '/' + fileName)
    
def generateImage(epoch, input_vector):
    """Generates an image based on a given input vector"""
    
    with torch.no_grad():
        # Generate image by passing randomized input vector to generator
        generated = G(input_vector)
        image = generated.view(generated.size(0), 1, 28, 28)
        
        now = datetime.now()
        fileName = str(now.strftime('%Y-%m-%dT%H:%M:%S')) + '_epoch_' + str(epoch) + '.png'
        
        if IN_COLAB:
            saveFileToGoogleDrive(image, fileName)
        else:
            saveFileToLocal(image, fileName)

In [ ]:
# Number of epochs
n_epoch = 2

# Input vector for image generation on trained network
input_vector = Variable(torch.randn(batch_size, image_dimension).to(device))

for epoch in range(1, n_epoch+1):           
    D_losses, G_losses = [], []
    for batch_idx, (x, _) in enumerate(train_loader):
        D_losses.append(D_train(x))
        G_losses.append(G_train(x))

    print('[%d/%d]: loss_d: %.3f, loss_g: %.3f' % (
            (epoch), n_epoch, torch.mean(torch.FloatTensor(D_losses)), torch.mean(torch.FloatTensor(G_losses))))
    generateImage(epoch, input_vector)
